Imports

In [ ]:
import json
import gspread
from google.colab import userdata
from oauth2client.service_account import ServiceAccountCredentials
import requests
from datetime import datetime
import pytz

In [ ]:
# Set up OpenWeatherMap API credentials
api_key = 'd8803f200694590d01f64d045af4efcf'
lat = 28.55
lon = -81.38

# Set up Google Sheets credentials
gcred = userdata.get('gcred')
gcred = json.loads(gcred) #Convert creds to JSON

# Authenticate with Google Sheets API
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_dict(gcred,scope)
client = gspread.authorize(credentials)


Define Function for unix conversion

In [ ]:
def convert_unix_to_eastern(unix):
    utc_time = datetime.utcfromtimestamp(unix)

    utc_time = utc_time.replace(tzinfo=pytz.utc) #Set the timezone for UTC

    eastern = pytz.timezone('US/Eastern') #Define the Eastern timezone

    eastern_time = utc_time.astimezone(eastern) #Convert the UTC time to Eastern Time

    formatted_time = eastern_time.strftime('%Y-%m-%d %H:%M') # Format the datetime as a string

    print(formatted_time)

    return formatted_time

Open and Clear Google Sheet

In [ ]:
spreadsheet_name = 'weather'

# Open the spreadsheet
spreadsheet = client.open(spreadsheet_name)

# Clear sheet
worksheet = spreadsheet.sheet1
worksheet.clear()

{'spreadsheetId': '1XpOArEsyNjReX-dwsPqDQAUyzoTFK2TS84ObEGDWGyA',
 'clearedRange': 'Sheet1!A1:Z1000'}

Get Current Weather

In [ ]:
# Get the current weather data
url = f'https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&appid={api_key}&units=imperial'
response = requests.get(url,timeout=30)
data = response.json()

# Extract the relevant information
current_weather = {
    'Date': convert_unix_to_eastern(data['current']['dt']),
    'Temperature': data['current']['temp'],
    'Humidity': data['current']['humidity'],
    'Description': data['current']['weather'][0]['description'],
    'Pressure': data['current']['pressure'],
    'WindSpeed': data['current']['wind_speed'],
    'WindDir': data['current']['wind_deg'],
    'Sunrise': convert_unix_to_eastern(data['current']['sunrise']),
    'Sunset': convert_unix_to_eastern(data['current']['sunset']),
    'FeelsLike' : data['current']['feels_like'],
    'UVIndex' : data['current']['uvi']
}

# Update the Google Sheet with the current weather data and time
worksheet.append_row(list(current_weather.values()))

2024-02-09 16:35
2024-02-09 07:08
2024-02-09 18:10


{'spreadsheetId': '1XpOArEsyNjReX-dwsPqDQAUyzoTFK2TS84ObEGDWGyA',
 'updates': {'spreadsheetId': '1XpOArEsyNjReX-dwsPqDQAUyzoTFK2TS84ObEGDWGyA',
  'updatedRange': 'Sheet1!A1:K1',
  'updatedRows': 1,
  'updatedColumns': 11,
  'updatedCells': 11}}

Get Forecast Data

In [ ]:
# Get the forecast data
url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={api_key}&units=imperial'
response = requests.get(url,timeout=30)
data = response.json()

# Extract the relevant forecast information
forecast = []
for forecast_item in data['list']:

    forecast_data = {
        'Date': convert_unix_to_eastern(forecast_item['dt']),
        'Temperature': forecast_item['main']['temp'],
        'Humidity': forecast_item['main']['humidity'],
        'Description': forecast_item['weather'][0]['description'],
        'Pressure' : forecast_item['main']['pressure'],
        'WindSpeed': forecast_item['wind']['speed'],
        'WindDir': forecast_item['wind']['deg'],
        'Sunrise': 0,
        'Sunset': 0,
        'FeelsLike' : forecast_item['main']['feels_like'],
        'UVIndex' : 0
    }
    forecast.append(forecast_data)

# Update the Google Sheet with the forecast data
for forecast_data in forecast:
    worksheet.append_row(list(forecast_data.values()))

print('Weather data and forecast have been stored in the Google Sheet.')

2024-02-09 19:00
2024-02-09 22:00
2024-02-10 01:00
2024-02-10 04:00
2024-02-10 07:00
2024-02-10 10:00
2024-02-10 13:00
2024-02-10 16:00
2024-02-10 19:00
2024-02-10 22:00
2024-02-11 01:00
2024-02-11 04:00
2024-02-11 07:00
2024-02-11 10:00
2024-02-11 13:00
2024-02-11 16:00
2024-02-11 19:00
2024-02-11 22:00
2024-02-12 01:00
2024-02-12 04:00
2024-02-12 07:00
2024-02-12 10:00
2024-02-12 13:00
2024-02-12 16:00
2024-02-12 19:00
2024-02-12 22:00
2024-02-13 01:00
2024-02-13 04:00
2024-02-13 07:00
2024-02-13 10:00
2024-02-13 13:00
2024-02-13 16:00
2024-02-13 19:00
2024-02-13 22:00
2024-02-14 01:00
2024-02-14 04:00
2024-02-14 07:00
2024-02-14 10:00
2024-02-14 13:00
2024-02-14 16:00
Weather data and forecast have been stored in the Google Sheet.
